# Kaggle: Dogs Vs Cats (using VGG16 pretrained network - finetuning)

In [1]:
import os,shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline

In [2]:
from keras.models import Sequential
from keras.layers import Input,Dropout,Flatten,Conv2D,Activation,MaxPooling2D,Dense
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint,Callback,EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array,load_img
from keras import applications

Using TensorFlow backend.


In [3]:
# since the data is already prepared for other runs, I am directly using those
train_data_dir = '/Users/ram/Desktop/DS-PRACTICE-PYTHON/DEEPNN/Cat_Vs_Dog/train_sub'
validation_data_dir = '/Users/ram/Desktop/DS-PRACTICE-PYTHON/DEEPNN/Cat_Vs_Dog/val_sub'
nb_train_samples = 2000
nb_validation_samples = 1100
epochs=50
batch_size=16
top_model_weights_path = 'Users/ram/Desktop/DS-PRACTICE-PYTHON/DEEPNN/Cat_Vs_Dog/bottleneck_fc_model.h5'

In [4]:
# build the VGG16 network
model_vgg16_base = applications.VGG16(include_top=False, weights='imagenet')

In [8]:
# Building a classifier to put on top of the conv model
# ------- need to check this ; a few errors are thrown--------
top_model = Sequential()
top_model.add(Flatten(input_shape=model_vgg16_base.output_shape[1:]))
top_model.add(Dense(256,activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1,activation='sigmoid'))

In [10]:
# it is necessary to start with a fully-trained classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

In [ ]:
# add the model on top of the convolutional base
model.add(top_model)

In [ ]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

In [ ]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)